# Ranking Functions
They are of three types:
1. Analytical Functions
2. Ranking Functions
3. Aggregate Functions

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T 
from pyspark.sql.window import Window 

In [2]:
import warnings
warnings.filterwarnings(action = 'ignore')

In [3]:
spark = SparkSession.builder.appName('Windows_Functions').getOrCreate()

24/01/04 21:54:06 WARN Utils: Your hostname, SAURAVs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.29.194 instead (on interface en0)
24/01/04 21:54:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/04 21:54:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
titanic = sns.load_dataset('titanic')
titanic_ps = spark.createDataFrame(titanic)


In [5]:
iris = sns.load_dataset('iris')
iris_ps = spark.createDataFrame(iris)


In [6]:
taxi = pd.read_csv('./datafiles/tripdata.csv', header=0)


In [7]:
taxi_ps = spark.read\
            .option('header', 'true')\
            .csv('/Users/sauravbhattacharyya/Documents/DataScience2022/PySpark/datafiles/tripdata.csv', inferSchema=True)

In [8]:
taxi.head(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,8/1/2019 0:09,8/1/2019 0:10,1,0.0,1,N,145,145,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8,0.0
1,1,8/1/2019 0:30,8/1/2019 0:31,1,0.0,1,N,145,145,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8,0.0
2,1,8/1/2019 0:44,8/1/2019 0:46,1,7.1,1,N,145,145,2,3.0,0.5,0.5,0.0,0.0,0.3,4.3,0.0


In [9]:
taxi.dtypes

VendorID                   int64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
trip_distance            float64
RatecodeID                 int64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type               int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object

In [10]:
taxi['rank'] = taxi['total_amount'].rank()
# method = average(default), first, min, max, dense

In [11]:
(25 + 26 + 27 + 28 + 29 + 30)/6

27.5

In [12]:
taxi[['total_amount', 'rank']].head(3)

,total_amount,rank
0,3.8,4859.0
1,3.8,4859.0
2,4.3,5989.0


In [13]:
taxi.sort_values(by='total_amount')[['total_amount', 'rank']].head(3)

,total_amount,rank
883781,-1871.8,1.0
496147,-452.8,2.0
123712,-149.3,3.0


In [14]:
taxi['first_rank'] = taxi['total_amount'].rank(method="first")

In [15]:
taxi[taxi['total_amount'] == -65.92].head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,rank,first_rank
126593,2,8/1/2019 16:04,8/1/2019 17:22,1,18.59,2,N,68,132,4,-52.0,-4.5,-0.5,0.0,-6.12,-0.3,-65.92,-2.5,27.5,25.0
158987,2,8/1/2019 18:15,8/1/2019 18:46,1,3.47,2,N,145,48,2,-52.0,-4.5,-0.5,0.0,-6.12,-0.3,-65.92,-2.5,27.5,26.0
380944,2,8/2/2019 18:33,8/2/2019 19:14,1,18.52,2,N,262,132,4,-52.0,-4.5,-0.5,0.0,-6.12,-0.3,-65.92,-2.5,27.5,27.0
422207,2,8/2/2019 21:04,8/2/2019 22:00,1,18.00,1,N,138,88,4,-56.0,-0.5,-0.5,0.0,-6.12,-0.3,-65.92,-2.5,27.5,28.0
915887,2,8/5/2019 16:54,8/5/2019 17:47,1,18.62,2,N,132,230,2,-52.0,-4.5,-0.5,0.0,-6.12,-0.3,-65.92,-2.5,27.5,29.0
919372,2,8/5/2019 16:03,8/5/2019 16:56,2,17.57,2,N,132,68,4,-52.0,-4.5,-0.5,0.0,-6.12,-0.3,-65.92,-2.5,27.5,30.0


In [16]:
taxi.sort_values(by='total_amount')[['total_amount', 'first_rank']].head(4)

,total_amount,first_rank
883781,-1871.80,1.0
496147,-452.80,2.0
123712,-149.30,3.0
751392,-147.04,4.0


In [17]:
taxi['min_rank'] = taxi['total_amount'].rank(method="min")

In [18]:
taxi.sort_values(by='total_amount')[['total_amount', 'min_rank']].tail(4)

,total_amount,min_rank
602291,760.02,1048572.0
1001846,799.06,1048573.0
981911,820.48,1048574.0
883782,1871.80,1048575.0


In [19]:
taxi['max_rank'] = taxi['total_amount'].rank(method="max")
taxi.sort_values(by='total_amount')[['total_amount', 'max_rank']].head(4)

,total_amount,max_rank
883781,-1871.80,1.0
496147,-452.80,2.0
123712,-149.30,3.0
751392,-147.04,4.0


In [20]:
taxi['dense_rank'] = taxi['total_amount'].rank(method="dense")


In [21]:
taxi.sort_values(by='total_amount')[['total_amount', 'dense_rank']].head(4)

,total_amount,dense_rank
883781,-1871.80,1.0
496147,-452.80,2.0
123712,-149.30,3.0
751392,-147.04,4.0


In [22]:
w = Window.orderBy("total_amount")
taxi_ps = taxi_ps.withColumn("row_number", F.row_number().over(w))


In [23]:
taxi_ps.select('total_amount', 'row_number').show(32)

24/01/04 21:54:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------+----------+
|total_amount|row_number|
+------------+----------+
|     -1871.8|         1|
|      -452.8|         2|
|      -149.3|         3|
|     -147.04|         4|
|      -140.3|         5|
|      -128.7|         6|
|      -122.8|         7|
|      -120.3|         8|
|      -104.8|         9|
|       -95.3|        10|
|       -94.3|        11|
|      -93.35|        12|
|       -85.8|        13|
|       -83.8|        14|
|       -80.3|        15|
|       -76.8|        16|
|       -75.8|        17|
|       -71.8|        18|
|       -68.8|        19|
|      -68.56|        20|
|       -68.3|        21|
|       -67.8|        22|
|      -66.42|        23|
|      -66.36|        24|
|      -65.92|        25|
|      -65.92|        26|
|      -65.92|        27|
|      -65.92|        28|
|      -65.92|        29|
|      -65.92|        30|
|       -65.8|        31|
|       -64.8|        32|
+------------+----------+
only showing top 32 rows



In [24]:
w = Window.orderBy("total_amount")
taxi_ps = taxi_ps.withColumn("dense_rank", F.dense_rank().over(w))


In [25]:
taxi_ps.select('total_amount', 'row_number', 'dense_rank').show(32)

24/01/04 21:54:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------+----------+----------+
|total_amount|row_number|dense_rank|
+------------+----------+----------+
|     -1871.8|         1|         1|
|      -452.8|         2|         2|
|      -149.3|         3|         3|
|     -147.04|         4|         4|
|      -140.3|         5|         5|
|      -128.7|         6|         6|
|      -122.8|         7|         7|
|      -120.3|         8|         8|
|      -104.8|         9|         9|
|       -95.3|        10|        10|
|       -94.3|        11|        11|
|      -93.35|        12|        12|
|       -85.8|        13|        13|
|       -83.8|        14|        14|
|       -80.3|        15|        15|
|       -76.8|        16|        16|
|       -75.8|        17|        17|
|       -71.8|        18|        18|
|       -68.8|        19|        19|
|      -68.56|        20|        20|
|       -68.3|        21|        21|
|       -67.8|        22|        22|
|      -66.42|        23|        23|
|      -66.36|        24|        24|
|

In [26]:
w = Window.orderBy("total_amount")
taxi_ps = taxi_ps.withColumn("rank", F.rank().over(w))


In [27]:
taxi_ps.select('total_amount', 'row_number', 'dense_rank', 'rank').show(32)

24/01/04 21:54:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 2

+------------+----------+----------+----+
|total_amount|row_number|dense_rank|rank|
+------------+----------+----------+----+
|     -1871.8|         1|         1|   1|
|      -452.8|         2|         2|   2|
|      -149.3|         3|         3|   3|
|     -147.04|         4|         4|   4|
|      -140.3|         5|         5|   5|
|      -128.7|         6|         6|   6|
|      -122.8|         7|         7|   7|
|      -120.3|         8|         8|   8|
|      -104.8|         9|         9|   9|
|       -95.3|        10|        10|  10|
|       -94.3|        11|        11|  11|
|      -93.35|        12|        12|  12|
|       -85.8|        13|        13|  13|
|       -83.8|        14|        14|  14|
|       -80.3|        15|        15|  15|
|       -76.8|        16|        16|  16|
|       -75.8|        17|        17|  17|
|       -71.8|        18|        18|  18|
|       -68.8|        19|        19|  19|
|      -68.56|        20|        20|  20|
|       -68.3|        21|        2

In [28]:
taxi_ps.show(3)

24/01/04 21:54:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 2

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+----------+----------+----+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|row_number|dense_rank|rank|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+----------+----------+----+
|       2|      8/5/2019 13:51|       8/8/2019 15:34|              1|          0.0|         3|                 N|         145|         145|   

In [29]:
taxi_ps.sort('total_amount')
w = Window.partitionBy('payment_type').orderBy('total_amount')
taxi_ps = taxi_ps.withColumn('new_row_num', F.row_number().over(w))


In [30]:
taxi_ps.where('payment_type = "1"')\
    .select('payment_type', 'total_amount', 'row_number', 'dense_rank', 'rank', 'new_row_num').show(30)

24/01/04 21:54:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------+------------+----------+----------+----+-----------+
|payment_type|total_amount|row_number|dense_rank|rank|new_row_num|
+------------+------------+----------+----------+----+-----------+
|           1|      -93.35|        12|        12|  12|          1|
|           1|      -17.16|       501|       129| 501|          2|
|           1|      -15.99|       518|       132| 518|          3|
|           1|        -6.8|      1654|       165|1621|          4|
|           1|         0.0|      2423|       181|2413|          5|
|           1|         0.0|      2424|       181|2413|          6|
|           1|         0.0|      2425|       181|2413|          7|
|           1|         0.0|      2427|       181|2413|          8|
|           1|         0.0|      2431|       181|2413|          9|
|           1|         0.0|      2432|       181|2413|         10|
|           1|         0.0|      2441|       181|2413|         11|
|           1|         0.0|      2443|       181|2413|        

In [31]:
taxi_ps.where('payment_type = "2"')\
    .select('payment_type', 'total_amount', 'row_number', 'dense_rank', 'rank', 'new_row_num').show(30)

24/01/04 21:54:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------+------------+----------+----------+----+-----------+
|payment_type|total_amount|row_number|dense_rank|rank|new_row_num|
+------------+------------+----------+----------+----+-----------+
|           2|      -149.3|         3|         3|   3|          1|
|           2|     -147.04|         4|         4|   4|          2|
|           2|      -104.8|         9|         9|   9|          3|
|           2|       -95.3|        10|        10|  10|          4|
|           2|       -83.8|        14|        14|  14|          5|
|           2|       -71.8|        18|        18|  18|          6|
|           2|      -66.42|        23|        23|  23|          7|
|           2|      -65.92|        26|        25|  25|          8|
|           2|      -65.92|        29|        25|  25|          9|
|           2|      -61.42|        36|        31|  36|         10|
|           2|      -61.42|        38|        31|  36|         11|
|           2|      -61.42|        39|        31|  36|        

In [32]:
taxi_ps.where('payment_type = "3"')\
    .select('payment_type', 'total_amount', 'row_number', 'dense_rank', 'rank', 'new_row_num').show(30)

24/01/04 21:54:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------+------------+----------+----------+----+-----------+
|payment_type|total_amount|row_number|dense_rank|rank|new_row_num|
+------------+------------+----------+----------+----+-----------+
|           3|      -452.8|         2|         2|   2|          1|
|           3|      -120.3|         8|         8|   8|          2|
|           3|       -80.3|        15|        15|  15|          3|
|           3|       -76.8|        16|        16|  16|          4|
|           3|       -75.8|        17|        17|  17|          5|
|           3|       -68.8|        19|        19|  19|          6|
|           3|      -68.56|        20|        20|  20|          7|
|           3|       -68.3|        21|        21|  21|          8|
|           3|       -65.8|        31|        26|  31|          9|
|           3|       -64.8|        32|        27|  32|         10|
|           3|      -63.42|        33|        28|  33|         11|
|           3|       -62.8|        34|        29|  34|        

In [33]:
taxi_ps.where('payment_type = "4"')\
    .select('payment_type', 'total_amount', 'row_number', 'dense_rank', 'rank', 'new_row_num').show(30)

24/01/04 21:54:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/04 21:54:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------+------------+----------+----------+----+-----------+
|payment_type|total_amount|row_number|dense_rank|rank|new_row_num|
+------------+------------+----------+----------+----+-----------+
|           4|     -1871.8|         1|         1|   1|          1|
|           4|      -140.3|         5|         5|   5|          2|
|           4|      -128.7|         6|         6|   6|          3|
|           4|      -122.8|         7|         7|   7|          4|
|           4|       -94.3|        11|        11|  11|          5|
|           4|       -85.8|        13|        13|  13|          6|
|           4|       -67.8|        22|        22|  22|          7|
|           4|      -66.36|        24|        24|  24|          8|
|           4|      -65.92|        25|        25|  25|          9|
|           4|      -65.92|        27|        25|  25|         10|
|           4|      -65.92|        28|        25|  25|         11|
|           4|      -65.92|        30|        25|  25|        

In [35]:
taxi_ps = spark.read\
            .option('header', 'true')\
            .csv('/Users/sauravbhattacharyya/Documents/DataScience2022/PySpark/datafiles/tripdata.csv', inferSchema=True)

In [43]:
w = Window.partitionBy('payment_type').orderBy(taxi_ps['total_amount'].desc())

In [45]:
taxi_ps.withColumn('ntile_rank', F.ntile(100).over(w))\
    .select('payment_type','total_amount', 'ntile_rank')\
    .show(3)

+------------+------------+----------+
|payment_type|total_amount|ntile_rank|
+------------+------------+----------+
|           1|      820.48|         1|
|           1|      799.06|         1|
|           1|      760.02|         1|
+------------+------------+----------+
only showing top 3 rows



In [49]:
taxi_ps.withColumn('percent_rank', F.percent_rank().over(w))\
    .select('payment_type','total_amount', 'percent_rank')\
    .show(3)

# (#Records more than the value)/(#Records in the corresponding partition - 1)

+------------+------------+--------------------+
|payment_type|total_amount|        percent_rank|
+------------+------------+--------------------+
|           1|      820.48|                 0.0|
|           1|      799.06|1.365584804864759...|
|           1|      760.02|2.731169609729518...|
+------------+------------+--------------------+
only showing top 3 rows



# Analytical Functions

In [53]:
w = Window.partitionBy('payment_type').orderBy(taxi_ps['total_amount'])
taxi_ps.withColumn("cume_dist",
                   F.cume_dist().over(w))\
                    .select('total_amount','cume_dist').show(10000, truncate=False)
# (#Records less than or equal to the value)/(#Records in the corresponding partition)

+------------+---------------------+
|total_amount|cume_dist            |
+------------+---------------------+
|-93.35      |1.3655829400454465E-6|
|-17.16      |2.731165880090893E-6 |
|-15.99      |4.09674882013634E-6  |
|-6.8        |5.462331760181786E-6 |
|0.0         |1.2563363048418108E-4|
|0.0         |1.2563363048418108E-4|
|0.0         |1.2563363048418108E-4|
|0.0         |1.2563363048418108E-4|
|0.0         |1.2563363048418108E-4|
|0.0         |1.2563363048418108E-4|
|0.0         |1.2563363048418108E-4|
|0.0         |1.2563363048418108E-4|
|0.0         |1.2563363048418108E-4|
|0.0         |1.2563363048418108E-4|
|0.0         |1.2563363048418108E-4|
|0.0         |1.2563363048418108E-4|
|0.0         |1.2563363048418108E-4|
|0.0         |1.2563363048418108E-4|
|0.0         |1.2563363048418108E-4|
|0.0         |1.2563363048418108E-4|
|0.0         |1.2563363048418108E-4|
|0.0         |1.2563363048418108E-4|
|0.0         |1.2563363048418108E-4|
|0.0         |1.2563363048418108E-4|
|

In [54]:
taxi_ps.withColumn("Lag",
                   F.lag('total_amount',2).over(w))\
                    .select('payment_type','total_amount','Lag').show(20)

+------------+------------+------+
|payment_type|total_amount|   Lag|
+------------+------------+------+
|           1|      -93.35|  null|
|           1|      -17.16|  null|
|           1|      -15.99|-93.35|
|           1|        -6.8|-17.16|
|           1|         0.0|-15.99|
|           1|         0.0|  -6.8|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
+------------+------------+------+
only showing top 20 rows



In [55]:
taxi_ps.withColumn("Lead",
                   F.lead('total_amount',2).over(w))\
                    .select('payment_type','total_amount','Lead').show(20)

+------------+------------+------+
|payment_type|total_amount|  Lead|
+------------+------------+------+
|           1|      -93.35|-15.99|
|           1|      -17.16|  -6.8|
|           1|      -15.99|   0.0|
|           1|        -6.8|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
|           1|         0.0|   0.0|
+------------+------------+------+
only showing top 20 rows



# Aggregate Functions

In [58]:
w = Window.partitionBy('payment_type')
taxi_ps = taxi_ps.withColumn("Avg", F.avg(F.col('total_amount')).over(w))
# F.sum, F.max, F.min 

In [61]:
taxi_ps.where('payment_type = "3"').show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|               Avg|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+------------------+
|       1|       8/1/2019 0:28|        8/1/2019 0:47|              1|         10.0|         4|                 N|         132|         265|           3|       39.5|  